In [1]:
import os
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from dotenv import load_dotenv

# Загружаем переменные окружения из .env (если требуется)
load_dotenv()

True

In [2]:
# Загрузка векторного хранилища (ChromaDB) из сохранённого каталога
# Здесь используются те же параметры, что применялись при загрузке данных
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

knowledge_vector_store = Chroma(
    collection_name="knowledge_markdown",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db/knowledge"
)

/var/folders/h2/r9wh0x750xq4v5z33ylwrzw40000gn/T/ipykernel_58409/1634373640.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/Users/sergey/Desktop/Voise_RAG/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Создание ретривера по схожести из векторного хранилища
retriever_similarity = knowledge_vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

In [4]:
# Извлечение всех документов из коллекции для построения BM25 ретривера
# Для этого используем внутренний метод ._collection.get() для доступа к сохранённым данным
raw_collection = knowledge_vector_store._collection.get()
docs = []
# В raw_collection ожидаются ключи "documents" и "metadatas"
for content, meta in zip(raw_collection["documents"], raw_collection["metadatas"]):
    docs.append(Document(page_content=content, metadata=meta))

# Создание BM25 ретривера из списка документов
keyword_retriever = BM25Retriever.from_documents(docs)
keyword_retriever.k = 5

In [5]:
# Объединение ретриверов через EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever_similarity, keyword_retriever],
    weights=[0.5, 0.5]
)

In [6]:
# Загрузка модели OpenRouter через модуль openrouter_loader
from openrouter_loader import load_model

# Загружаем модель (например, GPT-3.5-turbo) с нужными параметрами
generate_response = load_model(model_name="deepseek/deepseek-r1-distill-llama-8b", temperature=0.5, max_tokens=2048)

/Users/sergey/Desktop/Voise_RAG/Notebooks/openrouter_loader.py:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [7]:
# Пример функции для генерации ответа с использованием RAG (предполагается, что функция generate_answer уже определена)
def generate_answer(question: str, history: list = None) -> str:
    """
    Генерирует ответ на вопрос пользователя, используя комбинированный поиск по базе знаний.
    
    Шаги:
      1. Получение релевантных документов (ensemble_retriever.get_relevant_documents вызывает предупреждение о депрекации).
      2. Формирование контекста из найденных фрагментов.
      3. Составление prompt с контекстом и историей диалога (если предоставлена).
      4. Вызов LLM через OpenRouter для генерации ответа.
    """
    # Получение релевантных документов (обратите внимание, что метод get_relevant_documents устарел)
    relevant_docs = ensemble_retriever.get_relevant_documents(question)
    context_text = "\n\n".join([doc.page_content for doc in relevant_docs])
    
    if history:
        history_text = "\n".join(history)
        prompt = (
            "Тебе предоставлены следующие данные из базы знаний:\n\n"
            f"{context_text}\n\n"
            "История диалога:\n"
            f"{history_text}\n\n"
            "Используй эти данные для ответа на вопрос. Если информации недостаточно, сообщи об этом.\n\n"
            f"Вопрос: {question}"
        )
    else:
        prompt = (
            "Тебе предоставлены следующие данные из базы знаний:\n\n"
            f"{context_text}\n\n"
            "Используй эти данные для ответа на вопрос. Если информации недостаточно, сообщи об этом.\n\n"
            f"Вопрос: {question}"
        )
    
    # Вызов функции генерации ответа
    answer = generate_response(prompt)
    return answer

In [8]:
# Интерактивный чат в Jupyter Notebook
def chat():
    print("Начало чата. Для выхода введите 'х'.")
    history = []
    while True:
        user_input = input("Пользователь: ")
        if user_input.strip().lower() == 'х':
            print("Завершение чата.")
            break
        answer = generate_answer(user_input, history)
        print("\nБот:", answer, "\n")
        # Обновляем историю диалога: можно сохранять последние N обменов
        history.append(f"Пользователь: {user_input}")
        history.append(f"Бот: {answer}")

# Запуск чата
chat()

Начало чата. Для выхода введите 'х'.


/var/folders/h2/r9wh0x750xq4v5z33ylwrzw40000gn/T/ipykernel_58409/663165022.py:13: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = ensemble_retriever.get_relevant_documents(question)



Бот: Ошибка вызова модели: 'NoneType' object is not iterable 

Завершение чата.
